This is a summary and cleaner version of the jupyter lab called: http://127.0.0.1:8888/lab/tree/repo/autopi_inter/autopi_interneuron_decoding/notebooks/Get_data_for_modeing_4.ipynb

In this juper lab: 

1. load the projects with all the sessions
2. load cells and mvl around lever 
3. run lstmModel.py and import all functions
4. loop through all sessions with 1 or more than 1 FS and generate the shuffleing df.

Here I mostly wanted to load the result from the shuffeling and check if they worked for the sessions where the shuffling worked .. its in Shuffled_values forder

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import spikeA
from spikeA.Spike_train import Spike_train
from spikeA.Animal_pose import Animal_pose
from spikeA.Spatial_properties import Spatial_properties
from spikeA.Neuron import Simulated_place_cell, Simulated_grid_cell
from scipy.stats import poisson
from scipy.interpolate import interp1d
from tqdm import tqdm
from scipy import stats
from scipy import ndimage
from astropy.stats import circcorrcoef
from astropy import units as u
from functions import *
from lstmModel_earlystoping import *

In [2]:
%run ../../setup_project.py
%run ../../neuronAutopi.py

%run ../../generic_plot_functions.py

projectName, dataPath, dlcModelPath, myProject, sSessions = setup_project_session_lists(projectName="autopi_inter",
                                                                dataPath="/adata/projects/autopi_inter/",dlcModelPath="/adata/models")


creating myProject, an autopipy.project object
Project name: autopi_inter
dataPath: /adata/projects/autopi_inter/
dlcModelPath: /adata/models
Reading /adata/projects/autopi_inter//sessionList
We have 180 testing sessions in the list
spikeA.Kilosort_session objects are in sSessions


In [3]:
prepareSessionsForSpatialAnalysisProject(sSessions[:],myProject.sessionList[:]) ### sSeslist >>>> sSessions
#prepareSessionsForSpatialAnalysisProject(sSessions[152:154],myProject.sessionList[152:154]) ### sSeslist >>>> sSessions
#prepareSessionsForSpatialAnalysisProject(sSessions[130:153],myProject.sessionList[130:153]) ### sSeslist >>>> sSessions


Loading Animal_pose and Spike_train, sSes.ap and sSes.cg
Will load .pose.npy


100%|█████████████████████████████████████████| 180/180 [04:25<00:00,  1.47s/it]


Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


In [4]:
for ses, sSes in tqdm(zip(myProject.sessionList[:],sSessions[:])):
#for ses, sSes in tqdm(zip(myProject.sessionList[152:154],sSessions[152:154])):
#for ses, sSes in tqdm(zip(myProject.sessionList[130:153],sSessions[130:153])):
    getSearchHomingIntervals(ses, sSes)

180it [00:06, 26.59it/s]


In [5]:
%%time

fn=myProject.dataPath+"/results/cells_with_leverCells.csv"
print('loading', fn)
cells = pd.read_csv(fn)

## load the levertime press 
fn ='behavior_180_v2.csv'
print('loading', fn)
res = pd.read_csv(fn)
###
#df_HDInfoSingle = pd.read_csv('df_HDInfo.csv') 


loading /adata/projects/autopi_inter//results/cells_with_leverCells.csv
loading behavior_180_v2.csv
CPU times: user 143 ms, sys: 0 ns, total: 143 ms
Wall time: 170 ms


## Run the shuffling on all sessions:

In [6]:
from multiprocessing import Pool
from functools import partial
import os
from datetime import datetime
from tqdm import tqdm
import traceback
from concurrent.futures import ProcessPoolExecutor
from lstmModel_earlystoping import *
## SET THIS DIRECTORY ON lstmMODEL.py
save_directory ='/adata/projects/autopi_inter/results/interneurons_decoding_angleAroundlever/100_shuffle_Earlystoping/'
os.makedirs(save_directory, exist_ok=True)


iteration = 100


In [7]:
iteration

100

In [8]:
import gc
gc.collect()

0

+ This took 39 hours for 44 sessions for 2 different intervals .. in total 88 sessions around 39 hours


In [ ]:
from multiprocessing import Pool
from functools import partial
from datetime import datetime


# Track start time
now = datetime.now()
print("Start time:", now.strftime("%d/%m/%Y %H:%M:%S"))

# Define parameters
#interNames = ['atLever_light']
interNames = ['atLever_light', 'atLever_dark']

sigma_ifr = 5
patience = 20
factor = 0.6
iteration = 100
shuffle = True
verbose= False
save_directory = '/adata/projects/autopi_inter/results/interneurons_decoding_angleAroundlever/100_shuffle_Earlystoping/'

os.makedirs(save_directory, exist_ok=True)

# Define session pairs
#session_pairs = list(zip(sSessions[131:132], myProject.sessionList[131:132]))
session_pairs = list(zip(sSessions[:], myProject.sessionList[:]))

# Run in parallel using Pool
with Pool(4) as pool:
    process_func = partial(process_session_wrapper_earlystoping,interNames=interNames,
        cells=cells,sigma_ifr=sigma_ifr, patience=patience,
        factor=factor,iteration=iteration,
        shuffle=shuffle, verbose= verbose)
    
    pool.map(process_func, session_pairs)

# Track end time
now = datetime.now()
print("End time:", now.strftime("%d/%m/%Y %H:%M:%S"))


Start time: 02/12/2024 18:07:51
mn8578-05122021-0108


  0%|                                                   | 0/100 [00:00<?, ?it/s]

mn8578-29122021-0107


  0%|                                                   | 0/100 [00:00<?, ?it/s]

mn8599-03022022-0107


...

 34%|██████████████▎                           | 34/100 [18:31<32:07, 29.21s/it]

Early stopping triggered at epoch 21


 80%|█████████████████████████████████▌        | 80/100 [58:33<13:39, 41.00s/it]

Early stopping triggered at epoch 21


 35%|██████████████▋                           | 35/100 [18:56<30:22, 28.05s/it]

In [17]:
print(torch.__version__)      # PyTorch version
print(torch.version.cuda)     # CUDA version PyTorch was built with
print(torch.cuda.is_available())  # Check if CUDA is available

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = ""

1.11.0+cu102
10.2
True
